# **Install Libraries**

In [1]:
!pip install "ray[tune]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 44.1 MB/s eta 0:00:00


# **Import Libraries**

In [2]:
from filelock import FileLock
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from torch.utils.data import random_split
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from collections import OrderedDict

# **Data Loaders**

In [3]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    with FileLock(os.path.expanduser("~/.data.lock")):
        trainset = torchvision.datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transform
        )

        testset = torchvision.datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transform
        )

    return trainset, testset

# **Configurable Neural Network**

In [4]:
def flatten(x, start_dim=1, end_dim=-1):
    return x.flatten(start_dim=start_dim, end_dim=end_dim)

In [5]:
# Will need to modify the parameters and the network depending on what you are experimenting with

C, H, W = 3, 32, 32
num_classes = 10

channel_1 = 16
channel_2 = 32
channel_3 = 64

kernel_size_1 = 3
kernel_size_2 = 3
kernel_size_3 = 6

pad_size_1 = 2
pad_size_2 = 1
pad_size_3 = 1

pool_kernel_size = 2

fc_count_1 = 256

class Net(nn.Module):
    def __init__(self, learning_rate, momentum, weight_decay):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, channel_1, kernel_size_1, padding=pad_size_1)
        self.norm1 = nn.BatchNorm2d(channel_1)
        self.pool1 = nn.MaxPool2d(pool_kernel_size)
        self.conv2 = nn.Conv2d(channel_1, channel_2, kernel_size_2, padding=pad_size_2)
        self.norm2 = nn.BatchNorm2d(channel_2)
        self.pool2 = nn.MaxPool2d(pool_kernel_size)
        self.conv3 = nn.Conv2d(channel_2, channel_3, kernel_size_3, padding=pad_size_3)
        self.norm3 = nn.BatchNorm2d(channel_3)
        self.pool3 = nn.MaxPool2d(pool_kernel_size)        

        H_after_conv1 = int(H + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        W_after_conv1 = int(W + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        H_after_pool1 = int((H_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool1 = int((W_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        H_after_conv2 = int(H_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        W_after_conv2 = int(W_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        H_after_pool2 = int((H_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool2 = int((W_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)  
        H_after_conv3 = int(H_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        W_after_conv3 = int(W_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        H_after_pool3 = int((H_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool3 = int((W_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)

        self.fc1 = nn.Linear(H_after_pool3*W_after_pool3*channel_3, fc_count_1)
        self.fc2 = nn.Linear(fc_count_1, fc_count_1)
        self.fc3 = nn.Linear(fc_count_1, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = flatten(x)
        x = self.fc1(x)
        #x = F.relu(x)
        x = self.fc2(x)
        #x = F.relu(x)
        x = self.fc3(x)

        return x

Train using SGD optimizer

In [6]:
def train_cifar(config):
    net = Net(
      config['learning_rate'],
      config['momentum'],
      config['weight_decay']
    )  # Will need to modify the parameters depending on what you are experimenting with

    learning_rate = config['learning_rate']
    momentum = config['momentum']
    weight_decay = config['weight_decay']

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    optimizer = None

    criterion = nn.CrossEntropyLoss()

    if config['opt'] == 'SGD':
      if momentum == 0: 
        optimizer = optim.SGD(net.parameters(), lr=learning_rate, 
                      weight_decay=weight_decay,
                      momentum=0) 
      else: 
        optimizer = optim.SGD(net.parameters(), lr=learning_rate, 
                      weight_decay=weight_decay, 
                      momentum=momentum, nesterov=True)  
    elif config['opt'] == 'Adagrad':
      optimizer = optim.Adagrad(net.parameters(), lr=learning_rate, 
                    weight_decay=weight_decay)  
    elif config['opt'] == 'RMSprop':
      optimizer = optim.RMSprop(net.parameters(), lr=learning_rate, 
                    weight_decay=weight_decay, momentum=momentum)
    elif config['opt'] == 'Adam':
      optimizer = optim.Adam(net.parameters(), lr=learning_rate, 
                    weight_decay=weight_decay)

    # Do not modify any code below!
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(loaded_checkpoint_dir, "checkpoint.pt")
            )
            net.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=64, shuffle=True, num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=64, shuffle=True, num_workers=8
    )

    for epoch in range(10):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:
                print(
                    f"[{epoch + 1}, {i + 1:>5} loss: {running_loss / epoch_steps:.3f}]"
                )
                running_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        os.makedirs("my_model", exist_ok=True)
        torch.save((net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report(
            {"loss": (val_loss / val_steps), "accuracy": correct / total},
            checkpoint=checkpoint,
        )
    print("Finished Training")

# **Test Set Accuracy**

In [7]:
def test_best_model(best_result):
    best_trained_model = Net(
        best_result.config["learning_rate"], 
        best_result.config["momentum"], 
        best_result.config["weight_decay"]
     )  # Will need to modify the parameters depending on what you are experimenting with

    # Do not modify any code below!
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(
        best_result.checkpoint.to_directory(), "checkpoint.pt"
    )

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Best trial test set accuracy: {correct / total}")

# **Configuring the Search Space**

In [8]:
# You will be experimenting with the hyperparameters here
# Use tune.grid_search to specify what values you want to experiment with a hyperparameter

#with momentum
config = {"opt": tune.grid_search(['SGD', 'Adagrad', 'RMSprop', 'Adam']),
          "learning_rate": tune.grid_search([1e-3, 1e-2, 1e-1, 1]), 
          "momentum": tune.grid_search([0, 0.5, 0.9]), 
          "weight_decay": tune.grid_search([1e-4, 1e-3, 1e-2, 1e-1])}

In [9]:
# Do not modify any code below!
def main(config):
    #SGD
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar),
            resources={"gpu": 1},
        ),
        param_space=config,
    )
    results = tuner.fit()
    best_result = results.get_best_result("accuracy", "max")

    print(f"Best trial config: {best_result.config}")
    print(f"Best trial final validation loss: {best_result.metrics['loss']}")
    print(f"Best trial final validation accuracy: {best_result.metrics['accuracy']}")

    test_best_model(best_result)

    return results.get_dataframe("accuracy", "max")


results_df = main(config)

2023-04-13 14:34:42,905	INFO worker.py:1553 -- Started a local Ray instance.


(train_cifar pid=4790) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-13_14-34-41/train_cifar_551db_00000_0_learning_rate=0.0010,momentum=0,opt=SGD,weight_decay=0.0001_2023-04-13_14-34-44/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:02<00:00, 77591584.24it/s]


(train_cifar pid=4790) Extracting /root/ray_results/train_cifar_2023-04-13_14-34-41/train_cifar_551db_00000_0_learning_rate=0.0010,momentum=0,opt=SGD,weight_decay=0.0001_2023-04-13_14-34-44/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-13_14-34-41/train_cifar_551db_00000_0_learning_rate=0.0010,momentum=0,opt=SGD,weight_decay=0.0001_2023-04-13_14-34-44/data
(train_cifar pid=4790) Files already downloaded and verified


(train_cifar pid=4790) /usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
(train_cifar pid=4790)   warnings.warn(_create_warning_msg(


...

(train_cifar pid=4790) Extracting /root/ray_results/train_cifar_2023-04-13_14-34-41/train_cifar_551db_00191_191_learning_rate=1,momentum=0.9000,opt=Adam,weight_decay=0.1000_2023-04-13_19-44-19/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-13_14-34-41/train_cifar_551db_00191_191_learning_rate=1,momentum=0.9000,opt=Adam,weight_decay=0.1000_2023-04-13_19-44-19/data
(train_cifar pid=4790) Files already downloaded and verified
(train_cifar pid=4790) Finished Training


2023-04-13 19:45:54,218	INFO tune.py:798 -- Total run time: 18670.13 seconds (18669.83 seconds for the tuning loop).


Best trial config: {'opt': 'Adagrad', 'learning_rate': 0.01, 'momentum': 0.5, 'weight_decay': 0.01}
Best trial final validation loss: 0.7814221731416738
Best trial final validation accuracy: 0.7327


100%|██████████| 170498071/170498071 [00:02<00:00, 60040053.07it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Best trial test set accuracy: 0.6589


In [10]:
final_df = results_df.drop(
    columns=[
        "should_checkpoint",
        "timesteps_total",
        "episodes_total",
        "training_iteration",
        "trial_id",
        "experiment_id",
        "date",
        "timestamp",
        "pid",
        "hostname",
        "node_ip",
        "time_since_restore",
        "timesteps_since_restore",
        "iterations_since_restore",
        "warmup_time",
        "logdir",
    ]
)
final_df.sort_values(by=['accuracy'], ascending=False).head(10)


,loss,accuracy,time_this_iter_s,done,time_total_s,config/learning_rate,config/momentum,config/opt,config/weight_decay
113,0.781422,0.7327,8.496351,False,97.757634,0.010,0.5,Adagrad,0.0100
109,0.769208,0.7326,9.182420,False,96.530787,0.010,0.0,Adagrad,0.0100
76,0.790606,0.7315,8.074436,False,211.156329,0.001,0.5,RMSprop,0.0010
117,0.777699,0.7299,9.550015,False,87.284882,0.010,0.9,Adagrad,0.0100
9,0.808381,0.7279,8.945981,False,58.664941,0.010,0.9,SGD,0.0001
72,0.815173,0.7269,8.332555,False,59.553494,0.001,0.0,RMSprop,0.0010
54,0.835252,0.7266,8.275311,False,91.508422,0.100,0.5,SGD,0.0010
65,0.840388,0.7258,8.112665,False,68.425364,0.010,0.5,Adagrad,0.0010
84,0.894687,0.7245,9.771421,False,87.317663,0.001,0.0,Adam,0.0010
2,0.940317,0.7241,9.410465,False,98.588008,0.100,0.0,SGD,0.0001


In [11]:
final_df.sort_values(by=['accuracy']).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/learning_rate,config/momentum,config/opt,config/weight_decay
55,NaN,0.0971,14.265987,False,14.265987,1.0,0.5,SGD,0.001
107,NaN,0.0983,14.384753,False,14.384753,1.0,0.9,SGD,0.010
51,NaN,0.0990,14.267723,False,14.267723,1.0,0.0,SGD,0.001
99,NaN,0.0990,14.938224,False,14.938224,1.0,0.0,SGD,0.010
182,2.307736,0.0992,9.583983,False,101.261320,0.1,0.0,Adam,0.100
78,3338.569891,0.1007,14.054177,False,14.054177,0.1,0.5,RMSprop,0.001
59,NaN,0.1008,14.393024,False,14.393024,1.0,0.9,SGD,0.001
151,NaN,0.1015,15.736744,False,15.736744,1.0,0.5,SGD,0.100
183,2.363891,0.1016,8.585274,False,42.148058,1.0,0.0,Adam,0.100
155,NaN,0.1017,15.272421,False,15.272421,1.0,0.9,SGD,0.100
